In [1]:
# Run this cell to test if the LinearChain object is gonna give us a headache.
using IonSim

@time chain = LinearChain(
        ions=[Ca40([("S1/2", -1/2, "S"), ("D5/2", -1/2, "D")]), Ca40([("S1/2", -1/2, "S"), ("D5/2", -1/2, "D")])], 
        comfrequencies=(x=3e6,y=3e6,z=2.5e5), selectedmodes=(;z=[1],)
    )
chain = Nothing;

  7.907882 seconds (14.82 M allocations: 953.520 MiB, 7.61% gc time, 99.85% compilation time)


In [2]:
using QuantumOptics
using IonSim
import PyPlot
using Random, Distributions
Random.seed!(0)

include("./single_qubit_chamber.jl")
using .SingleQubitChamber: construct_single_qubit_chamber, RX, RY, RZ
const plt = PyPlot;

Globals

In [3]:
using .SingleQubitChamber: CALCIUM40 
TIMESCALE = 1e-6 
TRAP_FREQUENCY = 1e6
B_STRENGTH = 4e-4
PI_TIME = 2e-6;

More globals

In [4]:
@time chain = LinearChain(
        ions = [CALCIUM40], 
        comfrequencies = (x = 3e6, y = 3e6, z = TRAP_FREQUENCY), 
        selectedmodes = (;z = [1],)
    )

laser = Laser(Δ=0, ϵ = (x̂ - ẑ)/√2, k = (x̂ + ẑ)/√2, ϕ = 0)
chamber = Chamber(iontrap=chain, B=B_STRENGTH, Bhat=ẑ, δB=0, lasers=[laser]);

WAVELENGTH = transitionwavelength(CALCIUM40, ("S", "D"), chamber)
wavelength!(laser, WAVELENGTH)

INTENSITY = intensity_from_pitime(laser, PI_TIME, CALCIUM40, ("S", "D"), chamber);

Z_PLUS = CALCIUM40["S"]
VIB_MODE =  IonSim.modes(chamber)[1]
ψ0_MOTIONAL = VIB_MODE[0] # GLOBAL: ground state of the first vibrational mode

chain = Nothing
laser = Nothing
chamber = Nothing;

  1.627522 seconds (2.15 M allocations: 137.805 MiB, 2.79% gc time, 99.87% compilation time: 99% of which was recompilation)


Helper function for plotting

In [5]:
function plot_populations(chamber, tout, sol)
    vibrational_mode = modes(chamber)[1]

    Z_plus = CALCIUM40["S"]
    Z_minus = CALCIUM40["D"]

    X_plus = (Z_plus + Z_minus)/√2
    X_minus = (Z_plus - Z_minus)/√2

    Y_plus = (Z_plus + im*Z_minus)/√2
    Y_minus = (Z_plus - im*Z_minus)/√2

    prob_X_plus = expect(dm(X_plus) ⊗ one(vibrational_mode), sol)
    prob_X_minus = expect(dm(X_minus) ⊗ one(vibrational_mode), sol)
    prob_Y_plus = expect(dm(Y_plus) ⊗ one(vibrational_mode), sol)
    prob_Y_minus = expect(dm(Y_minus) ⊗ one(vibrational_mode), sol)
    prob_Z_plus = expect(dm(Z_plus) ⊗ one(vibrational_mode), sol)
    prob_Z_minus = expect(dm(Z_minus) ⊗ one(vibrational_mode), sol)

    fig, (x_ax, y_ax, z_ax) = plt.subplots(1, 3, figsize=(15, 5))
    x_ax.plot(tout, prob_X_plus, label="|X+⟩", linewidth=4)
    x_ax.plot(tout, prob_X_minus, "--", label="|X-⟩", linewidth=4)
    x_ax.set_xlim(tout[1], tout[end])
    x_ax.set_ylim(0, 1)
    x_ax.legend(loc=1)
    x_ax.set_xlabel("Time (μs)")

    y_ax.plot(tout, prob_Y_plus, label="|Y+⟩", linewidth=4)
    y_ax.plot(tout, prob_Y_minus, "--", label="|Y-⟩", linewidth=4)
    y_ax.set_xlim(tout[1], tout[end])
    y_ax.set_ylim(0, 1)
    y_ax.legend(loc=1)
    y_ax.set_xlabel("Time (μs)")

    z_ax.plot(tout, prob_Z_plus, label="|Z+⟩", linewidth=4)
    z_ax.plot(tout, prob_Z_minus, "--", label="|Z-⟩", linewidth=4)
    z_ax.set_xlim(tout[1], tout[end])
    z_ax.set_ylim(0, 1)
    z_ax.legend(loc=1)
    z_ax.set_xlabel("Time (μs)")

    return fig
end

plot_populations (generic function with 1 method)

To check that our $\texttt{find\_rabi\_freq}$ function works...

If it's doing what we think it does, then it should return a Rabi frequency which, when given to the built-in IonSim $\texttt{intensity\_from\_rabifrequency}$ function, should return the same value for intensity.

In [67]:
function find_rabi_freq(tout, sol, timescale)
    prob_Z_plus = expect(dm(Z_PLUS) ⊗ one(VIB_MODE), sol);
    prob_Z_plus_1up = broadcast(abs, prob_Z_plus[2:end]);
    max_prob_idx = findall(broadcast(abs, prob_Z_plus_1up.-maximum(prob_Z_plus_1up)) .< 1e-8)[1];
    period = tout[max_prob_idx+1]-tout[1];
    period *= timescale
    freq = 1/period
    return freq
end

chamber = construct_single_qubit_chamber(
    TRAP_FREQUENCY,
    INTENSITY,
    WAVELENGTH,
    PI_TIME,
    B_STRENGTH
)

# Initial state
ψ0_electronic = CALCIUM40["S"]
ψ0 = Z_PLUS ⊗ ψ0_MOTIONAL# GLOBAL: initial state for all the simulations

# RX(2π)
tout, ψt = RX(2π, chamber, ψ0, PI_TIME, TIMESCALE);

omega_func = find_rabi_freq(tout, ψt, TIMESCALE)
print("Function output: ", intensity_from_rabifrequency(chamber.lasers[1], omega_func, CALCIUM40, ("S", "D"), chamber))
print("\nOriginal intensity: ", INTENSITY)


Function output: 4.123759471808808e6
Original intensity: 4.123759471808808e6

Cool, so it works. Now, we can estimate $\sigma_I$.




$\sigma_\Omega^2 = E[\Omega^2] - E^2[\Omega]$